# Set up spark

In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
from IPython import get_ipython

# set the application name as "<your_gaspar_id>-homework3"
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-final", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [3]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-final -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7357,application_1652960972356_3072,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
%%spark
import pyspark.sql.functions as functions

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Get main stops

In [39]:
%%spark
stops = spark.read.option("header",True).csv('/user/sixu/work/stops_main.csv')
stops = stops.drop("_c0")
stops.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- stop_name: string (nullable = true)
 |-- stop_lat: string (nullable = true)
 |-- stop_lon: string (nullable = true)
 |-- main_id: string (nullable = true)

In [40]:
%%spark
stops.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+--------+-------+
|           stop_name| stop_lat|stop_lon|main_id|
+--------------------+---------+--------+-------+
|Zimmerberg-Basist...|47.351677|8.521957|    176|
|Oetwil a.d.L., Sc...|47.423626|8.403183|8500926|
|Zürich Flughafen,...|47.451023|8.563729|8502075|
|Dietikon Stoffelbach|47.393326| 8.39896|8502186|
|Rudolfstetten Hof...| 47.36467|8.376952|8502187|
+--------------------+---------+--------+-------+
only showing top 5 rows

# Get trips

In [14]:
%%spark
stop_times = spark.read.orc('/data/sbb/part_orc/stop_times')
stop_times.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_sequence: short (nullable = true)
 |-- pickup_type: byte (nullable = true)
 |-- drop_off_type: byte (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)

In [45]:
%%spark
from pyspark.sql import Row

# filter by date
stop_times_515 = stop_times.filter(stop_times.year=='2019')\
        .filter(stop_times.month=='5')\
        .filter(stop_times.day=='15')

# process id
stop_times_515 = stop_times_515.rdd.map(lambda x: (
    x.trip_id,
    x.stop_id.lstrip('Parent').split(":")[0].rstrip('P'),
    x.arrival_time,
    x.departure_time,
    x.stop_sequence,
    )).toDF(["trip_id","stop_id","arrival_time","departure_time","stop_sequence"])
stop_times_515.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+------------+--------------+-------------+
|             trip_id|stop_id|arrival_time|departure_time|stop_sequence|
+--------------------+-------+------------+--------------+-------------+
|1.TA.1-1-B-j19-1.1.R|8500010|    04:20:00|      04:20:00|            1|
|1.TA.1-1-B-j19-1.1.R|8500020|    04:24:00|      04:24:00|            2|
|1.TA.1-1-B-j19-1.1.R|8500021|    04:28:00|      04:28:00|            3|
|1.TA.1-1-B-j19-1.1.R|8517131|    04:30:00|      04:30:00|            4|
|1.TA.1-1-B-j19-1.1.R|8500300|    04:32:00|      04:32:00|            5|
+--------------------+-------+------------+--------------+-------------+
only showing top 5 rows

<font color="red">11128930 tuples left</font>

In [49]:
%%spark
# filter by stops in radius
stop_times_515 = stop_times_515.join(stops, stop_times_515.stop_id==stops.main_id, "inner")
stop_times_515.drop("main_id")
stop_times_515.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+------------+--------------+-------------+------------------+---------+--------+-------+
|             trip_id|stop_id|arrival_time|departure_time|stop_sequence|         stop_name| stop_lat|stop_lon|main_id|
+--------------------+-------+------------+--------------+-------------+------------------+---------+--------+-------+
|5.TA.1-1-A-j19-1.3.H|8503305|    02:42:00|      02:42:00|            2|        Effretikon|47.425816|8.686682|8503305|
|5.TA.1-1-A-j19-1.3.H|8503306|    02:46:00|      02:46:00|            3|         Dietlikon|  47.4202|8.619272|8503306|
|5.TA.1-1-A-j19-1.3.H|8503147|    02:50:00|      02:50:00|            4|         Stettbach|47.397213|8.596141|8503147|
|5.TA.1-1-A-j19-1.3.H|8503003|    02:55:00|      02:55:00|            5|Zürich Stadelhofen| 47.36661|8.548485|8503003|
|5.TA.1-1-A-j19-1.3.H|8503000|    02:58:00|      03:00:00|            6|         Zürich HB|47.378178|8.540212|8503000|
+--------------------+-------+------------+-----

<font color="red">2307009 tuples left</font>

In [52]:
# %%spark
# stop_times_515_in_radius.write.option("header",True).csv("/user/sixu/work/stop_times_515_main.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Handle duplicate

In [53]:
%%spark
temp = stop_times_515.filter(stop_times_515.stop_id == '8503305')
temp.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+------------+--------------+-------------+----------+---------+--------+-------+
|             trip_id|stop_id|arrival_time|departure_time|stop_sequence| stop_name| stop_lat|stop_lon|main_id|
+--------------------+-------+------------+--------------+-------------+----------+---------+--------+-------+
|5.TA.1-1-A-j19-1.3.H|8503305|    02:42:00|      02:42:00|            2|Effretikon|47.425816|8.686682|8503305|
|6.TA.1-1-A-j19-1.4.H|8503305|    03:42:00|      03:42:00|            2|Effretikon|47.425816|8.686682|8503305|
|7.TA.1-1-A-j19-1.5.H|8503305|    03:42:00|      03:42:00|            2|Effretikon|47.425816|8.686682|8503305|
|1.TA.26-1-A-j19-1...|8503305|    01:21:00|      01:21:00|            5|Effretikon|47.425816|8.686682|8503305|
|2.TA.26-1-A-j19-1...|8503305|    01:21:00|      01:21:00|            5|Effretikon|47.425816|8.686682|8503305|
|3.TA.26-1-A-j19-1...|8503305|    01:21:00|      01:21:00|            5|Effretikon|47.425816|8.686682|8503305|
|

<font color="red">How can we know which trip_id?
Maybe we can get timetable from actual data directly...</color>

# Actural data?

In [88]:
%%spark
sbb_515 = spark.read.orc('/data/sbb/orc/istdaten')
sbb_515 = sbb_515.filter(sbb_515.betriebstag=='15.05.2019').filter(sbb_515.zusatzfahrt_tf=='false')
sbb_515.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

<font color="red">1386907 tuples left</color>

In [91]:
%%spark
# filter by stops in radius
sbb_515 = sbb_515.join(stops, sbb_515.haltestellen_name==stops.stop_name, "inner")
sbb_515.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

318049

<font color="red">Only 318049 tuples!</color>

In [85]:
%%spark
sbb_515.select('fahrt_bezeichner').show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|fahrt_bezeichner|
+----------------+
|    85:11:10:001|
|  85:11:1007:001|
|  85:11:1009:001|
|  85:11:1011:001|
| 85:11:10190:001|
| 85:11:10190:001|
| 85:11:10540:001|
| 85:11:10541:001|
| 85:11:10542:002|
| 85:11:10737:001|
| 85:11:10738:001|
| 85:11:10739:001|
| 85:11:10740:001|
| 85:11:10836:001|
| 85:11:10836:001|
| 85:11:10838:001|
| 85:11:10838:001|
| 85:11:10839:001|
| 85:11:10839:001|
| 85:11:10841:001|
+----------------+
only showing top 20 rows

In [86]:
%%spark -o selected_df
id_list = ['85:11:10:001', '85:11:10190:001', '85:11:10541:001', '85:11:10838:001', '85:11:10839:001']
selected_df = stop_times_515.filter(stop_times_515.trip_id.isin(id_list))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
selected_df.count()

Series([], dtype: int64)

<font color="red">Trip ids in actual data and stop_times don't overlap!</color>

# Plot trip

In [96]:
%%spark -o abb_515_m
# only keep necessary data and rename
abb_515_m = sbb_515.rdd.map(lambda x:(
    x.fahrt_bezeichner,
    x.haltestellen_name,
    x.faellt_aus_tf,
    x.ankunftszeit,
    x.an_prognose,
    x.abfahrtszeit,
    x.ab_prognose,
    x.durchfahrt_tf
    )).toDF(["trip_id","stop_name","failed","arrival_schedule","arrival_actual","departure_schedule","departure_actual","not_stop"])

abb_515_m.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------+------+----------------+-------------------+------------------+-------------------+--------+
|       trip_id|stop_name|failed|arrival_schedule|     arrival_actual|departure_schedule|   departure_actual|not_stop|
+--------------+---------+------+----------------+-------------------+------------------+-------------------+--------+
|  85:11:10:001|Zürich HB| false|15.05.2019 21:50|15.05.2019 21:50:28|                  |                   |   false|
|85:11:1007:001|Zürich HB| false|15.05.2019 06:23|15.05.2019 06:23:06|                  |                   |   false|
|85:11:1009:001|Zürich HB| false|15.05.2019 07:23|15.05.2019 07:22:45|                  |                   |   false|
|85:11:1011:001|Zürich HB| false|15.05.2019 08:23|15.05.2019 08:24:08|                  |                   |   false|
|  85:11:11:001|Zürich HB| false|                |                   |  15.05.2019 06:10|15.05.2019 06:11:54|   false|
+--------------+---------+------+---------------

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
%%spark -o stops
stops.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+--------+-------+
|           stop_name| stop_lat|stop_lon|main_id|
+--------------------+---------+--------+-------+
|Zimmerberg-Basist...|47.351677|8.521957|    176|
|Oetwil a.d.L., Sc...|47.423626|8.403183|8500926|
|Zürich Flughafen,...|47.451023|8.563729|8502075|
|Dietikon Stoffelbach|47.393326| 8.39896|8502186|
|Rudolfstetten Hof...| 47.36467|8.376952|8502187|
+--------------------+---------+--------+-------+
only showing top 5 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
# prepare data
token = "pk.eyJ1IjoiY29jb251dG51dCIsImEiOiJjbDNscTZhbHowMmxtM2pwajl3Yjd1ejF0In0.PXbwkPmWYXrAhQsus3ypVA"

trip_list = abb_515_m['trip_id'].unique().tolist()
trip_list.sort()
len(trip_list)

332

In [98]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import HBox, VBox

In [118]:
# token = "pk.eyJ1IjoiY29jb251dG51dCIsImEiOiJjbDNscTZhbHowMmxtM2pwajl3Yjd1ejF0In0.PXbwkPmWYXrAhQsus3ypVA"
# px.set_mapbox_access_token(token)
# fig = px.line_mapbox(selected_df, lat="stop_lat", lon="stop_lon", color="trip_id")
# fig.show()

# figure
layout = dict(hovermode='closest',
    margin=dict(l=0, t=0, r=0, b=0, pad=0),
    mapbox=dict(
        accesstoken=token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=47.378177,
            lon=8.540192
        ),
        pitch=0,
        zoom=10
    ))

data = go.Scattermapbox(
        lat=[],
        lon=[],
        mode = "markers+lines",
        marker=go.scattermapbox.Marker(
            size=5
        ),
        text=[]
    )


figure = go.FigureWidget(data=[data], layout=layout)

# update function
def f(a):
    trip = abb_515_m[abb_515_m['trip_id'] == a]
    # get lon and lat
    trip = trip.join(stops.set_index('stop_name'), on='stop_name')
    
    # update figure
    with figure.batch_update():
        figure.data[0].lat = trip['stop_lat']
        figure.data[0].lon = trip['stop_lon']
        figure.data[0].text = trip['stop_name']
        
# widget
choose_stop = widgets.Dropdown(
    description='Stop:',
    options=trip_list
)

out = widgets.interactive_output(f, {'a': choose_stop})

# display
VBox([HBox([choose_stop, out]), figure])

# Actural data 514

In [99]:
%%spark
sbb_514 = spark.read.orc('/data/sbb/orc/istdaten')
sbb_514 = sbb_514.filter(sbb_514.betriebstag=='14.05.2019').filter(sbb_514.zusatzfahrt_tf=='false')
sbb_514.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

In [100]:
%%spark
stop_name_514 = sbb_514.select('haltestellen_name').distinct()
stop_name_514.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10241

In [101]:
%%spark
stop_name_514 = stop_name_514.join(stops, stop_name_514.haltestellen_name==stops.stop_name, "inner")
stop_name_514.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1413

<font color="red">There are 10241 distinct stop names, and 1413 in 15km radius.</font>

In [102]:
%%spark
sbb_514 = sbb_514.join(stops, sbb_514.haltestellen_name==stops.stop_name, "inner")
sbb_514.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

317981

In [103]:
%%spark
sbb_514.select('haltestellen_name').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1413

In [104]:
%%spark
sbb_514.select('fahrt_bezeichner').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24438

<font color="red">There are 317981 tuples, 1413 distinct stops, and 24438 distinct trips.</font>

In [105]:
%%spark
sbb_514_m = sbb_514.select("haltestellen_name", "fahrt_bezeichner","faellt_aus_tf","ankunftszeit","an_prognose","abfahrtszeit","ab_prognose","durchfahrt_tf")

# rename
newColumns = ["stop_name2","trip_id","failed","arrival_schedule","arrival_actual","departure_schedule","departure_actual","not_stop"]
sbb_514_m = sbb_514_m.toDF(*newColumns)
sbb_514_m.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------+------+----------------+-------------------+------------------+-------------------+--------+
|stop_name2|       trip_id|failed|arrival_schedule|     arrival_actual|departure_schedule|   departure_actual|not_stop|
+----------+--------------+------+----------------+-------------------+------------------+-------------------+--------+
| Zürich HB|  85:11:10:001| false|14.05.2019 21:50|14.05.2019 21:52:07|                  |                   |   false|
| Zürich HB|85:11:1007:001| false|14.05.2019 06:23|14.05.2019 06:22:54|                  |                   |   false|
| Zürich HB|85:11:1009:001| false|14.05.2019 07:23|14.05.2019 07:23:39|                  |                   |   false|
| Zürich HB|85:11:1011:001| false|14.05.2019 08:23|14.05.2019 08:23:24|                  |                   |   false|
| Zürich HB|  85:11:11:001| false|                |                   |  14.05.2019 06:10|14.05.2019 06:11:14|   false|
| Zürich HB|  85:11:12:001| false|14.05.

In [106]:
%%spark
# get stop id
sbb_514_m = sbb_514_m.join(stops, sbb_514_m.stop_name2==stops.stop_name, "inner")
sbb_514_m = sbb_514_m.drop("stop_name2")
sbb_514_m.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

317981

In [107]:
%%spark
sbb_514_m.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------+----------------+-------------------+------------------+-------------------+--------+---------+---------+--------+-------+
|       trip_id|failed|arrival_schedule|     arrival_actual|departure_schedule|   departure_actual|not_stop|stop_name| stop_lat|stop_lon|main_id|
+--------------+------+----------------+-------------------+------------------+-------------------+--------+---------+---------+--------+-------+
|  85:11:10:001| false|14.05.2019 21:50|14.05.2019 21:52:07|                  |                   |   false|Zürich HB|47.378178|8.540212|8503000|
|85:11:1007:001| false|14.05.2019 06:23|14.05.2019 06:22:54|                  |                   |   false|Zürich HB|47.378178|8.540212|8503000|
|85:11:1009:001| false|14.05.2019 07:23|14.05.2019 07:23:39|                  |                   |   false|Zürich HB|47.378178|8.540212|8503000|
|85:11:1011:001| false|14.05.2019 08:23|14.05.2019 08:23:24|                  |                   |   false|Zürich HB|47.378

<font color="red">This is reasonable</font>

# Get and output data

In [115]:
%%spark
def getDataOnDate(date):
    # read data
    sbb = spark.read.orc('/data/sbb/orc/istdaten')
    # filter by date and not additional trip
    sbb = sbb.filter(sbb.betriebstag==date).filter(sbb.zusatzfahrt_tf=='false')
    # keep stops in 15km radius
    sbb = sbb.join(stops, sbb.haltestellen_name==stops.stop_name, "inner")
    
    # select necessary columns
    sbb = sbb.select("haltestellen_name", "fahrt_bezeichner","faellt_aus_tf","ankunftszeit","an_prognose","abfahrtszeit","ab_prognose","durchfahrt_tf")
    # rename
    newColumns = ["stop_name2","trip_id","failed","arrival_schedule","arrival_actual","departure_schedule","departure_actual","not_stop"]
    sbb = sbb.toDF(*newColumns)
    
    # join and get stop_id
    sbb = sbb.join(stops, sbb.stop_name2==stops.stop_name, "inner")
    sbb = sbb.drop("stop_name2")
    sbb = sbb.withColumnRenamed("main_id", "stop_id")
    
    return sbb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
%%spark
sbb_517 = getDataOnDate('17.05.2019')
sbb_517.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

326126

In [128]:
%%spark
sbb_517.write.option("header",True).csv("/user/sixu/work/actual_data_517.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
%%spark
sbb_517.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------+----------------+-------------------+------------------+-------------------+--------+---------+---------+--------+-------+
|       trip_id|failed|arrival_schedule|     arrival_actual|departure_schedule|   departure_actual|not_stop|stop_name| stop_lat|stop_lon|stop_id|
+--------------+------+----------------+-------------------+------------------+-------------------+--------+---------+---------+--------+-------+
|  85:11:10:001| false|17.05.2019 21:50|17.05.2019 21:51:14|                  |                   |   false|Zürich HB|47.378178|8.540212|8503000|
|85:11:1007:001| false|17.05.2019 06:23|17.05.2019 06:23:20|                  |                   |   false|Zürich HB|47.378178|8.540212|8503000|
|85:11:1009:001| false|17.05.2019 07:23|17.05.2019 07:23:31|                  |                   |   false|Zürich HB|47.378178|8.540212|8503000|
|85:11:1011:001| false|17.05.2019 08:23|17.05.2019 08:24:16|                  |                   |   false|Zürich HB|47.378